<h1>SPAM EMAIL CLASSIFIER</h1>

In [50]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

class Spam_Email_Classifier:

    # LEVEL 1 {The Analyst(Data Preprocessing)}
    def data_loading(self,file_excel,file_csv):
        file=pd.read_excel(file_excel,header=None)
        file.to_csv(file_csv,sep="\t",index=False,header=False)
        file=pd.read_csv(file_csv,sep="\t",names=["Spam/Ham","Email"])
        return file

    def data_cleaning(self,file):
        file=file.dropna()
        file=file.drop_duplicates(keep="first")
        return file

    def text_preprocessing(self,text):
        text=text.lower()
        text=re.sub("[^a-zA-Z0-9]"," ",text)
        return text

    # LEVEL 2 {The Builder(Feature Engineering & Model)}
    def vectorizer(self,file,file_preprocessed_data,file_label):
        peak_accuracy=0.00005*(len(file)**2)+0.05*len(file)+700
        peak_accuracy=int(peak_accuracy)
        TFIDF=TfidfVectorizer(max_features=peak_accuracy,stop_words="english")
        x=TFIDF.fit_transform(file_preprocessed_data).toarray()
        y=pd.get_dummies(file_label).iloc[:,1].values
        return x,y,TFIDF

    def Train_Test_split(self,x,y,ratio):
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=ratio,random_state=42)
        return x_train,x_test,y_train,y_test

    def model_training(self,x_train,x_test,y_train):
        model=MultinomialNB()
        model.fit(x_train,y_train)
        y_result=model.predict(x_test)
        return model,y_result

    # LEVEL 3 {The Evaluator(Metrics & Validation)}
    def Classification_report(self,y_test,y_result):
        print("Classification Report:")
        print(classification_report(y_test,y_result))

    def custom_test(self,model,TFIDF):
        S=input("Enter your Email:")
        S_preprocessed=self.text_preprocessing(S)
        S_transform=TFIDF.transform([S_preprocessed])
        S_predict=model.predict(S_transform)
        if S_predict[0]:
            print("ALERT:Spam Email")
        else:
            print("Safe Email")

    # main()
    def main(self):
        fh=self.data_loading("Spam_Email_Dataset.xlsx","Spam_Email_Dataset.csv")
        fh=self.data_cleaning(fh)
        fh["Preprocessed Email"]=fh["Email"].apply(self.text_preprocessing)
        x,y,TFIDF=self.vectorizer(fh,fh["Preprocessed Email"],fh["Spam/Ham"])
        x_train,x_test,y_train,y_test=self.Train_Test_split(x,y,0.2)
        model,y_result=self.model_training(x_train,x_test,y_train)
        self.Classification_report(y_test,y_result)
        self.custom_test(model,TFIDF)

activate=Spam_Email_Classifier()
activate.main()

Classification Report:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99       894
        True       0.98      0.88      0.93       140

    accuracy                           0.98      1034
   macro avg       0.98      0.94      0.96      1034
weighted avg       0.98      0.98      0.98      1034



Enter your Email: Congratulations! You Won $1000


ALERT:Spam Emial
